<h1><center>DS Capstone Project - Barcelona vs Paris - Let the battle begin.</center></h1><h3><center>Barcelona vs Paris - How (dis)similar are they?</center></h3>

<h4>1. Problem and discussion of the background.</h4>

Barcelona and Paris are two of my most favourite cities in Europe. But why is it that I like these cities the most? Are they similar or very dissimilar. Both cities have different backgrounds, but what makes them so popular. Let's find out if we can use data to solve my problem and to find out which city should be chosen for the next visit.

<h4>2. How will data be used to solve the problem. </h4>

Now we have chosen to use data to find out the (dis)similarities between the two cities, we need to decide which/what data to use. Let's use data from Wikipedia and FourSquare and find out if they can help us to find the answer!
To specify our quest a little bit more we will look for venues and more specifically the number of Vietnamese, Chinese, and Japanese restaurants in both cities as well as list down the 10 most common venues in both cities Neighbourhood wise. I have chosen these types of cuisines as I have an South-East Asian background. Foursquare is a perfect provider to supply us this data.
Wikipedia will be used for getting a start on geospatial information on the districts of Barcelona and arrondissements of Paris.
The outcome of the study will be used to determine to which city the next visit will be organized.

<h4>3. Let the battle begin. </h4>

In [ ]:
! pip install geopy

In [ ]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.expand_frame_repr', False)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

In [ ]:
## Foursquare ID
CLIENT_ID = 'NDGX4RBYCYKMWHJNX2MALLNW4ETTWAXUP3D54BN3N5MEGBVJ' # your Foursquare ID
CLIENT_SECRET = 'LUXBWPXTLFIEYSOGH4JI5TM4MBWS5010VBHZ2J243HQJMC10' # your Foursquare Secret
VERSION = '20200630' # Foursquare API version

#Fourshare Variables
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

venue_category_1 = '4bf58dd8d48988d14a941735' # Vietnamese restaurant 
venue_category_2 = '4bf58dd8d48988d145941735' # Chinese restaurant
venue_category_3 = '4bf58dd8d48988d111941735' # Japanese restaurant

category_list = ['Vietnamese Restaurant' , 'Chinese Restaurant', 'Japanese Restaurant']

In [ ]:
#Retrieve the hierarchical category list from FourSquare
venue_categories_url = 'https://api.foursquare.com/v2/venues/categories?client_id={}&client_secret={}&v={}'.format(CLIENT_ID,CLIENT_SECRET,VERSION)

#venue_categories_url
!wget -q -O 'venue_categories_hierarchy.json' "https://api.foursquare.com/v2/venues/categories?client_id=NDGX4RBYCYKMWHJNX2MALLNW4ETTWAXUP3D54BN3N5MEGBVJ&client_secret=LUXBWPXTLFIEYSOGH4JI5TM4MBWS5010VBHZ2J243HQJMC10&v=20180605"
#!wget -q -O 'venue_categories_hierarchy.json' 
print('Data downloaded!')


In [ ]:
venue_results = requests.get(venue_categories_url).json()["response"]['categories'][3]

venue_results_main = pd.json_normalize(data=venue_results)
venue_results_main = venue_results_main.rename(columns={'name': 'MainCategoryName'})
venue_results_main = venue_results_main.drop(columns=['categories', 'pluralName', 'shortName', 'icon.prefix', 'icon.suffix'])

venue_results_1 = pd.json_normalize(data=venue_results, record_path=['categories'], meta=['id'], record_prefix='l1_')
venue_results_1 = venue_results_1.drop(columns=['l1_categories','l1_pluralName', 'l1_shortName', 'l1_icon.prefix', 'l1_icon.suffix'])

venue_results_2 = pd.json_normalize(data=venue_results, record_path=['categories','categories'], meta=[['name','id']], record_prefix='l2_')
venue_results_2 = venue_results_2.rename(columns={'name.id': 'CategoryId_l1', 'l2_id':'CategoryId_l2', 'l2_name':'CategoryName_l2'})
venue_results_2 = venue_results_2.drop(columns=['l2_categories','l2_pluralName', 'l2_shortName', 'l2_icon.prefix', 'l2_icon.suffix'])

venue_results_3 = pd.json_normalize(data=venue_results, record_path=['categories','categories','categories'], meta=[['name', 'name','id']], record_prefix='l3_')
venue_results_3 = venue_results_3.rename(columns={'name.name.id': 'CategoryId_l2', 'l3_id':'CategoryId_l3', 'l3_name':'CategoryName_l3'})
venue_results_3 = venue_results_3.drop(columns=['l3_categories','l3_pluralName', 'l3_shortName', 'l3_icon.prefix', 'l3_icon.suffix'])

venue_results_4 = pd.json_normalize(data=venue_results, record_path=['categories','categories','categories','categories'], meta=[['name','name', 'name','id']], record_prefix='l4_')
venue_results_4 = venue_results_4.rename(columns={'name.name.name.id': 'CategoryId_l3', 'l4_id':'CategoryId_l4', 'l4_name':'CategoryName_l4'})
venue_results_4 = venue_results_4.drop(columns=['l4_categories','l4_pluralName', 'l4_shortName', 'l4_icon.prefix', 'l4_icon.suffix'])

venue_categories = pd.merge(venue_results_main, venue_results_1, how='inner', on='id')
venue_categories = venue_categories.rename(columns={'id': 'MainCategoryId', 'l1_id': 'CategoryId_l1', 'l1_name': 'CategoryName_L1' } )

venue_categories = pd.merge(venue_categories, venue_results_2, how='left', left_on='CategoryId_l1', right_on='CategoryId_l1')
venue_categories = pd.merge(venue_categories, venue_results_3, how='left', left_on='CategoryId_l2', right_on='CategoryId_l2')
venue_categories = pd.merge(venue_categories, venue_results_4, how='left', left_on='CategoryId_l3', right_on='CategoryId_l3')
venue_categories = venue_categories.drop(columns=['MainCategoryId','CategoryId_l1', 'CategoryId_l2', 'CategoryId_l3', 'CategoryId_l4'])

venue_categories.CategoryName_l2.fillna(venue_categories.CategoryName_L1, inplace=True)
venue_categories.CategoryName_l3.fillna(venue_categories.CategoryName_l2, inplace=True)
venue_categories.CategoryName_l4.fillna(venue_categories.CategoryName_l3, inplace=True)
venue_categories.head()

In [ ]:
# Function to get all nearby venues given the geo coordinates of a place.
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
# Function to get all nearby venues which match the given Category Id's and given the geo coordinates of a place.
def getNearbyVenuesCat(names, latitudes, longitudes, radius=500):
    
    venues_list_cat=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url_cat = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={},{},{}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            venue_category_1,
            venue_category_2,
            venue_category_3)
            
        # make the GET request
        results_cat = requests.get(url_cat).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list_cat.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results_cat])

    nearby_venues_cat = pd.DataFrame([item for venues_list_cat in venues_list_cat for item in venues_list_cat])
    nearby_venues_cat.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues_cat)

<h4>3.1 Show Barcelona and the boroughs on the map. </h4>

Paris has in total 10 boroughs (called districts) and 75 neighborhoods. To explore, analyze and segment neighborhoods, longitude and latitude of each neighbor and borough will be added.
The data is stored in a CSV file and its content is retrieved through Wikipedia.

In [ ]:
!wget -q -O 'barcelona_districts_geodata.csv'  "https://raw.githubusercontent.com/AQuynhVo/Quynh_Repo/master/DS_Capstone_Project/barcelona_districts_geodata.csv"
print('Data downloaded!')

In [ ]:
bcn_data = pd.read_csv('barcelona_districts_geodata.csv', sep=';')

bcn_data.head()

In [ ]:
print('The Barcelona dataframe has {} boroughs (Districts) and {} neighbourhoods.'.format(
        len(bcn_data['District'].unique()),
        bcn_data.shape[0]
    )
)

In [ ]:
bcn_address = 'Barcelona, ES'

geolocator = Nominatim(user_agent="bcn_explorer")
bcn_location = geolocator.geocode(bcn_address)
bcn_latitude = bcn_location.latitude
bcn_longitude = bcn_location.longitude
print('The geograpical coordinate of Barcelona (ES) are {}, {}.'.format(bcn_latitude, bcn_longitude))

In [ ]:
# create map of Barcelona using latitude and longitude values
map_barcelona = folium.Map(location=[bcn_latitude, bcn_longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighbourhood in zip(bcn_data['Latitude'], bcn_data['Longitude'], bcn_data['District'], bcn_data['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_barcelona)  
    
    
# display map
map_barcelona

<h4>3.2 Using Foursquare to discover Barcelona Venues. </h4>

<h4>3.2.1 Using Foursquare to discover Barcelona Venues from all kinds of categories</h4>

In [ ]:
barcelona_venues = getNearbyVenues(names=bcn_data['Neighbourhood'],
                                   latitudes=bcn_data['Latitude'],
                                   longitudes=bcn_data['Longitude']
                                  )

In [ ]:
print(barcelona_venues.shape)
barcelona_venues.head()
#barcelona_venues.groupby('Neighborhood').count()

In [ ]:
# Let's find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories in Barcelona.'.format(len(barcelona_venues['Venue Category'].unique())))

<h4>3.2.1 Using Foursquare to discover Barcelona Venues by having a look at the selected categories. </h4>

In [ ]:
barcelona_venues_sel_cat = getNearbyVenuesCat(names=bcn_data['Neighbourhood'],
                                   latitudes=bcn_data['Latitude'],
                                   longitudes=bcn_data['Longitude']
                                  )

In [ ]:
print(barcelona_venues_sel_cat.shape)
barcelona_venues_sel_cat

barcelona_venues_sel_cat_ext = pd.merge(barcelona_venues_sel_cat, venue_categories, how='left', left_on='Venue Category', right_on='CategoryName_l3')

barcelona_venues_sel_cat_ext = barcelona_venues_sel_cat_ext.drop(columns=['MainCategoryName', 'CategoryName_L1', 'CategoryName_l3', 'CategoryName_l4'])

barcelona_venues_sel_cat_ext.CategoryName_l2.fillna(barcelona_venues_sel_cat_ext['Venue Category'], inplace=True)

barcelona_venues_sel_cat_ext = barcelona_venues_sel_cat_ext.rename(columns={'CategoryName_l2': 'Venue Final Category'})

print(barcelona_venues_sel_cat_ext.head())
print(barcelona_venues_sel_cat_ext.shape)

In [ ]:
#Filter out records not in the category list.
barcelona_venues_sel_cat_ext = barcelona_venues_sel_cat_ext[barcelona_venues_sel_cat_ext['Venue Final Category'].isin(category_list)]
print(barcelona_venues_sel_cat_ext.shape)

In [ ]:
# Number of restaurants per Neighbourhood and Venue Final Category
Barcelona_City_Stats_Selected_Categories = barcelona_venues_sel_cat_ext.groupby( ['Neighborhood', 'Venue Final Category'] ).size().to_frame(name = 'count').reset_index()
Barcelona_City_Stats_Selected_Categories = pd.merge(Barcelona_City_Stats_Selected_Categories, bcn_data, how='left', left_on='Neighborhood', right_on='Neighbourhood')
Barcelona_City_Stats_Selected_Categories

In [ ]:
#df.pivot(index='foo', columns='bar', values='baz')
Barcelona_City_Stats_Selected_Categories_Pivoted = pd.pivot_table(Barcelona_City_Stats_Selected_Categories, index=['District','Neighborhood'], columns='Venue Final Category', values='count')
Barcelona_City_Stats_Selected_Categories_Pivoted

In [ ]:
#import matplotlib
#import matplotlib.pyplot as plt

custom_colors = ['#5cb85c','#5bc0de','#d9534f']

ax = Barcelona_City_Stats_Selected_Categories_Pivoted.plot(kind='bar', stacked=False, figsize=(20, 8), color=custom_colors, fontsize=14, width = 0.8 )
ax.set_title('Number of Venues per Barcelona District & Neighbourhood', fontsize=16)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
plt.legend(fontsize=14)
plt.tick_params(left=False, labelleft=False)
   
for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x()+.13, p.get_height() +1.5), va="center", ha="center")
#plt.show()

In [ ]:
# add Venue markers to map
for lat, lng, venue, venuecategory in zip(barcelona_venues_sel_cat['Venue Latitude'], barcelona_venues_sel_cat['Venue Longitude'], barcelona_venues_sel_cat['Venue'], barcelona_venues_sel_cat['Venue Category']):
    label = '{}, {}'.format(venue, venuecategory)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_barcelona)  
    
    
# display map
map_barcelona

<h4>3.3 Show Paris and the boroughs on the map. </h4>

Paris has in total 20 boroughs (called arrondissements in French) and 80 neighborhoods. To explore, analyze and segment neighborhoods, longitude and latitude of each neighbor and borough will be added.
This dataset exists for free on the web.

Links to the dataset are:
- https://www.data.gouv.fr/fr/datasets/r/e88c6fda-1d09-42a0-a069-606d3259114e
- https://opendata.paris.fr/explore/dataset/quartier_paris/download/?format=json&timezone=Europe/Berlin

In [ ]:
!wget -q -O 'paris_data.json'  "https://opendata.paris.fr/explore/dataset/quartier_paris/download/?format=json&timezone=Europe/Berlin"
print('Data downloaded!')

In [ ]:
with open('paris_data.json') as paris_json_data:
    paris_data = json.load(paris_json_data)

##paris_data

In [ ]:
# Tranform the data into a *pandas* dataframe
# The next task is essentially transforming this data of nested Python dictionaries into a *pandas* dataframe. So let's start by creating an empty dataframe.
# paris_data[1]

# define the dataframe columns
column_names = ['Borough', 'Neighbourhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
paris_neighbourhoods = pd.DataFrame(columns=column_names)

for data in paris_data:
    borough = neighborhood_name = data['fields']['c_ar'] 
    neighborhood_name = data['fields']['l_qu']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    paris_neighbourhoods = paris_neighbourhoods.append({'Borough': borough,
                                          'Neighbourhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    
paris_neighbourhoods.head()

In [ ]:
print('The Paris dataframe has {} boroughs and {} neighbourhoods.'.format(
        len(paris_neighbourhoods['Borough'].unique()),
        paris_neighbourhoods.shape[0]
    )
)

In [ ]:
par_address = 'Paris, FR'

par_geolocator = Nominatim(user_agent="par__explorer")
par_location = par_geolocator.geocode(par_address)
par_latitude = par_location.latitude
par_longitude = par_location.longitude
print('The geograpical coordinate of Paris (FR) are {}, {}.'.format(par_latitude, par_longitude))

In [ ]:
# create map of Toronto using latitude and longitude values
map_paris = folium.Map(location=[par_latitude, par_longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighbourhood in zip(paris_neighbourhoods['Latitude'], paris_neighbourhoods['Longitude'], paris_neighbourhoods['Borough'], paris_neighbourhoods['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_paris)  
    
map_paris

<h4>3.4 Using Foursquare to discover Paris Venues. </h4>

<h4>3.4.1 Using Foursquare to discover Paris Venues from all kinds of categories</h4>

In [ ]:
paris_venues = getNearbyVenues(names=paris_neighbourhoods['Neighbourhood'],
                                   latitudes=paris_neighbourhoods['Latitude'],
                                   longitudes=paris_neighbourhoods['Longitude']
                                  )

In [ ]:
print(paris_venues.shape)
paris_venues.head()
#paris_venues.groupby('Neighborhood').count()



In [ ]:
#### Let's find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories in Paris.'.format(len(paris_venues['Venue Category'].unique())))

<h4>3.4.2 Using Foursquare to discover Paris Venues by having a look at the selected categories. </h4>

In [ ]:
paris_venues_sel_cat = getNearbyVenuesCat(names=paris_neighbourhoods['Neighbourhood'],
                                   latitudes=paris_neighbourhoods['Latitude'],
                                   longitudes=paris_neighbourhoods['Longitude']
                                  )

In [ ]:
print(paris_venues_sel_cat.shape)

#barcelona_venues_sel_cat.groupby('Neighborhood').count()

paris_venues_sel_cat_ext = pd.merge(paris_venues_sel_cat, venue_categories, how='left', left_on='Venue Category', right_on='CategoryName_l3')

paris_venues_sel_cat_ext = paris_venues_sel_cat_ext.drop(columns=['MainCategoryName', 'CategoryName_L1', 'CategoryName_l3', 'CategoryName_l4'])

paris_venues_sel_cat_ext.CategoryName_l2.fillna(paris_venues_sel_cat_ext['Venue Category'], inplace=True)

paris_venues_sel_cat_ext = paris_venues_sel_cat_ext.rename(columns={'CategoryName_l2': 'Venue Final Category'})

print(paris_venues_sel_cat_ext.head())
print(paris_venues_sel_cat_ext.shape)

In [ ]:
#Filter out records not in the category list.
paris_venues_sel_cat_ext = paris_venues_sel_cat_ext[paris_venues_sel_cat_ext['Venue Final Category'].isin(category_list)]
print(paris_venues_sel_cat_ext.shape)

In [ ]:
# Number of restaurants per Neighbourhood and Venue Final Category
Paris_City_Stats_Selected_Categories = paris_venues_sel_cat_ext.groupby( ['Neighborhood', 'Venue Final Category'] ).size().to_frame(name = 'count').reset_index()
Paris_City_Stats_Selected_Categories = pd.merge(Paris_City_Stats_Selected_Categories, paris_neighbourhoods, how='left', left_on='Neighborhood', right_on='Neighbourhood')
Paris_City_Stats_Selected_Categories

In [ ]:
#df.pivot(index='foo', columns='bar', values='baz')
Paris_City_Stats_Selected_Categories_Pivoted = pd.pivot_table(Paris_City_Stats_Selected_Categories, index=['Borough','Neighborhood'], columns='Venue Final Category', values='count')
Paris_City_Stats_Selected_Categories_Pivoted

In [ ]:
#import matplotlib
#import matplotlib.pyplot as plt

custom_colors = ['#5cb85c','#5bc0de','#d9534f']

ax = Paris_City_Stats_Selected_Categories_Pivoted.plot(kind='bar', stacked=False, figsize=(20, 8), color=custom_colors, fontsize=10, width = 0.8 )
ax.set_title('Number of Venues per Paris Arrondissement & Neighbourhood', fontsize=16)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
plt.legend(fontsize=14)
plt.tick_params(left=False, labelleft=False)
   
for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x()+.13, p.get_height() +1.5), va="center", ha="center")
#plt.show()

In [ ]:
paris_venues_sel_cat_ext['Venue Category'].unique()

In [ ]:
# add Venue markers to map
for lat, lng, venue, venuecategory in zip(paris_venues_sel_cat_ext['Venue Latitude'], paris_venues_sel_cat['Venue Longitude'], paris_venues_sel_cat['Venue'], paris_venues_sel_cat['Venue Category']):
    label = '{}, {}'.format(venue, venuecategory)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_paris)  
    
    
# display map
map_paris